# The Movie DataBase (TMDB)

In [62]:
import pandas as pd
import numpy as np
import json
import re

In [90]:
movies= pd.read_csv('tmdb_5000_movies.csv')
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [57]:
credits= pd.read_csv('tmdb_5000_credits.csv')
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


## Normalizzazione

#### Creo tabella movie_id

In [20]:
movie_id = movies
del movie_id['keywords']
del movie_id['genres']
del movie_id['production_companies']
del movie_id['production_countries']
del movie_id['spoken_languages']
movie_id = movie_id.set_index(['id'])
movie_id.head(2)

,budget,homepage,original_language,original_title,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count
id,,,,,,,,,,,,,,
19995,237000000,http://www.avatarmovie.com/,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,2787965087,162.0,Released,Enter the World of Pandora.,Avatar,7.2,11800
285,300000000,http://disney.go.com/disneypictures/pirates/,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,961000000,169.0,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


#### Creo tabella genere

In [19]:
genere=pd.DataFrame()
for film in list(range(0,len(movies['genres']))):
    tabjson=pd.read_json(movies['genres'][film])
    genere=pd.concat([genere, tabjson])
genere = genere.set_index(['id'])
genere= genere.drop_duplicates() #per eliminare le righe duplicate
genere.head()

,name
id,
28,Action
12,Adventure
14,Fantasy
878,Science Fiction
80,Crime


#### Creo tabella genere_id

In [23]:
genere_id=pd.DataFrame()
for film in list(range(0,len(movies['genres']))):
    tabjson=pd.read_json(movies['genres'][film])
    idfilm=movies['id'][film]
    tabjson['movie_id']=idfilm
    genere_id=pd.concat([genere_id, tabjson])
del genere_id['name']
genere_id = genere_id.set_index(['movie_id', 'id'])
genere_id.head(10)

Empty DataFrame
Columns: []
Index: [(19995, 28.0), (19995, 12.0), (19995, 14.0), (19995, 878.0), (285, 12.0), (285, 14.0), (285, 28.0), (206647, 28.0), (206647, 12.0), (206647, 80.0)]

#### Creo tabella keywords

In [24]:
keywords=pd.DataFrame()
for film in list(range(0,len(movies['keywords']))):
    tabjson=pd.read_json(movies['keywords'][film])
    keywords=pd.concat([keywords, tabjson])
keywords = keywords.set_index(['id'])
keywords = keywords.drop_duplicates()
keywords.head()

,name
id,
1463,culture clash
2964,future
3386,space war
3388,space colony
3679,society


#### Creo tabella keywords_id

In [167]:
keywords_id=pd.DataFrame()
for film in list(range(0,len(movies['keywords']))):
    tabjson=pd.read_json(movies['keywords'][film])
    idfilm=movies['id'][film]
    tabjson['movie_id']=idfilm
    keywords_id=pd.concat([keywords_id, tabjson])
del keywords_id['name']
keywords_id = keywords_id.set_index(['movie_id', 'id'])
keywords_id.head()

Empty DataFrame
Columns: []
Index: [(19995, 1463.0), (19995, 2964.0), (19995, 3386.0), (19995, 3388.0), (19995, 3679.0)]

#### Creo tabella prod_companies

In [20]:
prod_companies=pd.DataFrame()
for film in list(range(0,len(movies['production_companies']))):
    tabjson=pd.read_json(movies['production_companies'][film])
    prod_companies=pd.concat([prod_companies, tabjson])
prod_companies = prod_companies.set_index(['id'])
prod_companies = prod_companies.drop_duplicates()
prod_companies.head()

,name
id,
289,Ingenious Film Partners
306,Twentieth Century Fox Film Corporation
444,Dune Entertainment
574,Lightstorm Entertainment
2,Walt Disney Pictures


#### Creo tabella prod_companies_id

In [170]:
prod_companies_id=pd.DataFrame()
for film in list(range(0,len(movies['production_companies']))):
    tabjson=pd.read_json(movies['production_companies'][film])
    idfilm=movies['id'][film]
    tabjson['movie_id']=idfilm
    prod_companies_id=pd.concat([prod_companies_id, tabjson])
del prod_companies_id['name']
prod_companies_id = prod_companies_id.set_index(['movie_id', 'id'])
prod_companies_id.head()

Empty DataFrame
Columns: []
Index: [(19995, 289.0), (19995, 306.0), (19995, 444.0), (19995, 574.0), (285, 2.0)]

#### Creo tabella prod_countries

In [21]:
prod_countries=pd.DataFrame()
for film in list(range(0,len(movies['production_countries']))):
    tabjson=pd.read_json(movies['production_countries'][film])
    prod_countries=pd.concat([prod_countries, tabjson])
prod_countries = prod_countries.set_index(['iso_3166_1'])
prod_countries = prod_countries.drop_duplicates()
prod_countries.head()

,name
iso_3166_1,
US,United States of America
GB,United Kingdom
JM,Jamaica
BS,Bahamas
DM,Dominica


#### Creo tabella prod_countries_id

In [173]:
prod_countries_id=pd.DataFrame()
for film in list(range(0,len(movies['production_countries']))):
    tabjson=pd.read_json(movies['production_countries'][film])
    idfilm=movies['id'][film]
    tabjson['movie_id']=idfilm
    prod_countries_id=pd.concat([prod_countries_id, tabjson])
del prod_countries_id['name']
prod_countries_id = prod_countries_id.set_index(['movie_id', 'iso_3166_1'])
prod_countries_id.head()

Empty DataFrame
Columns: []
Index: [(19995, US), (19995, GB), (285, US), (206647, GB), (206647, US)]

#### Creo tabella spoken_languages

In [26]:
spoken_languages=pd.DataFrame()
for film in list(range(0,len(movies['spoken_languages']))):
    tabjson=pd.read_json(movies['spoken_languages'][film])
    spoken_languages=pd.concat([spoken_languages, tabjson])
spoken_languages = spoken_languages.set_index(['iso_639_1'])
spoken_languages = spoken_languages.drop_duplicates()
spoken_languages.head()

,name
iso_639_1,
en,English
es,Español
fr,Français
it,Italiano
de,Deutsch
tr,Türkçe
el,ελληνικά
zh,普通话
th,ภาษาไทย


#### Creo tabella spoken_languages_id

In [182]:
spoken_languages_id=pd.DataFrame()
for film in list(range(0,len(movies['spoken_languages']))):
    tabjson=pd.read_json(movies['spoken_languages'][film])
    idfilm=movies['id'][film]
    tabjson['movie_id']=idfilm
    spoken_languages_id=pd.concat([spoken_languages_id, tabjson])
del spoken_languages_id['name']
spoken_languages_id = spoken_languages_id.set_index(['movie_id', 'iso_639_1'])
spoken_languages_id.head()

Empty DataFrame
Columns: []
Index: [(19995, en), (19995, es), (285, en), (206647, fr), (206647, en)]

1- For each movie, compute the number of cast members

In [58]:
def cast_members(data):
    item_dict = json.loads(data)
    return(len(item_dict))
credits['cast_members'] = credits['cast'].apply(cast_members)
credits[['movie_id','cast_members']].head()

,movie_id,cast_members
0,19995,83
1,285,34
2,206647,83
3,49026,158
4,49529,27


2- How many movies do not have a homepage?

In [60]:
nohomepage=movies[movies['homepage'].isnull()]
len(nohomepage.index)

3091

3- For each year, how many movies do not have a homepage?

In [118]:
def extract_year(date):
    if (pd.isnull(date)):
        year=np.nan
    else:
        data = datetime.datetime.strptime(date, "%Y-%m-%d")
        year=data.year
    return(year)
movies['years'] = movies['release_date'].apply(extract_year)
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,years
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,2009.0
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,2007.0


In [132]:
movies['years'] = movies['years'].replace(np.nan, 'Missing')
nohomepage=movies[movies['homepage'].isnull()]
nohomepage[['id', 'years']].groupby('years').count().tail()

,id
years,
2013.0,127
2014.0,157
2015.0,110
2016.0,31
Missing,1




4- Extract the domain of each homepage.

5- Extract a set of normalized tables. That is, each entry of a normalized table must contain exactly one value (not a list or a dictionary).

6- For each movie, compute the gross margin (difference between revenue and budget)

7- For each movie, compute the number of crew members

8- For each movie, compute the number of directors

9- For each language, compute the number of movies where such language is spoken.

10- For each company and each decade, compute the overall revenue

11- For each decade, compute the company with maximum revenue

12- In each year, how many movies have revenue smaller than the budget?



1- Distribute the revenue according to the order of appearance in a movie. Assume that the i-th actor contributes twice as much as the (i+1)-th actor to the revenue.

2- For each actor find the total revenue attributed to him/her.

3- Find the actor that is responsible for the most overall revenue.



1- For each movie, compute the ratio between males and females in the cast

2- For each movie, compute the ratio between the attributed revenue of males and females in the cast

3- Find the director that has the highest average ratio computed in the previous point.
